
# Nonlinear equations

In [ ]:
from functools import partial

import pandas as pd
import numpy as np


from algorithms_nonlinear import bisect
from plots_nonlinear import plot_bisect_example

from algorithms_nonlinear import fixpoint
from plots_nonlinear import plot_fixpoint_example

from algorithms_nonlinear import newton_method
from problems_nonlinear import get_cournot_problem

from algorithms_nonlinear import mcp_minmax, mcp_fischer
from problems_nonlinear import get_mcp_problem
from problems_nonlinear import get_spacial_market
from problems_nonlinear import get_fischer_problem

from scipy import optimize

## Bisection method

We can start with some special cases to develop the basic building blocks for more complicated material

In [ ]:
def f(x):
    return x ** 3 - 2


a, b = 1, 2
bisect(f, a, b)

In [ ]:
plot_bisect_example(f, a, b)

## Function iteration

In [ ]:
def f(x):
    return np.sqrt(x)


fixpoint(f, 2)

In [ ]:
plot_fixpoint_example(f)

## Newton's method

In [ ]:
def f(x):
    return x ** 3 - 2, 3 * x ** 2


newton_method(f, np.array([1.0]))

In [ ]:
c, e = np.array([0.6, 0.8]), 1.6
cournot_p = partial(get_cournot_problem, c, e)

newton_method(cournot_p, np.array([0.2, 0.2]))

## Quasi-Newton methods

In [ ]:
c, e = np.array([0.6, 0.8]), 1.6
cournot_p = partial(get_cournot_problem, c, e, jac=False)

optimize.root(cournot_p, np.array([0.8, 0.2]), method="broyden1")

In [ ]:
cournot_p = partial(get_cournot_problem, c, e)
newton_method(cournot_p, np.array([0.2, 0.2]))

## Convergence rates

## Benchmarking exercise

We now consider a more challenging task and compare the performance of `scipy`'s root finding algorithms.

In [ ]:
def exponential_function(x):

    p = x.shape[0]
    rslt = np.tile(np.nan, p)

    for i in range(p):
        if i == 0:
            rslt[i] = np.exp(x[i]) - 1
        else:
            rslt[i] = (i / 10) * (np.exp(x[i]) + x[i - 1] - 1)

    return rslt


METHODS = ["broyden1", "broyden2", "anderson", "Krylov"]

OPTIONS = dict()
OPTIONS["maxiter"] = 500

DIMENSION = 10

In [ ]:
df = pd.DataFrame(columns=["Algorithm", "Sample", "Success", "Iteration"])

In [ ]:
counter = 0
options = dict()
for method in METHODS:

    summary = dict()
    summary["success"] = 0
    for _ in range(10):

        counter += 1
        x0 = np.random.uniform(size=DIMENSION)

        try:
            rslt = optimize.root(exponential_function, x0, method=method, options=options,)
        except OverflowError:
            rslt = dict()
            rslt["success"] = False
            rslt["nit"] = OPTIONS["maxiter"]

        summary["success"] += rslt["success"]

        df.loc[counter] = [method, _, rslt["success"], rslt["nit"]]

## Nonlinear complementarity problems

In [ ]:
x0 = np.array([0.5, 0.5])
a, b = np.array([[0.0, 0.0], [1.0, 1.0]])
mcp_minmax(get_mcp_problem, x0, a, b)

## Here I get different values for x than in the presentations, but the implied quantities and prices are the same.

In [ ]:
a = np.zeros(9)
b = np.full(9, np.inf)
x0 = np.zeros(9)
x_sol = mcp_minmax(get_spacial_market, x0, a, b)
quantities = x_sol["x"].reshape(3, 3).round(4)

In [ ]:
if True:
    A = np.array
    as_ = A([9, 3, 18])
    bs = A([1, 2, 1])
    ad = A([42, 54, 51])
    bd = A([2, 3, 1])
    c = A([[0, 3, 9], [3, 0, 3], [6, 3, 0.0]])

prices = as_ + bs * quantities.sum(0)
prices

In [ ]:
exports = quantities.sum(0) - quantities.sum(1)

In [ ]:
a = np.zeros(1)
# need to make robust
b = np.full(1, np.inf)
x0 = np.zeros(1)

x_sol = mcp_minmax(get_fischer_problem, x0, a, b)
print("Minmax", x_sol)

x_sol = mcp_fischer(get_fischer_problem, x0, a, b)
print("Fischer", x_sol)